In [1]:
import pandas as pd
import numpy as np
import os
import imageio
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Maximum
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Add
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize as imresize
from tqdm import tqdm
import matplotlib.pyplot as plt


from subprocess import check_output

In [2]:
# Dense layers set
def dense_set(inp_layer, n, activation, drop_rate=0.):
    dp = Dropout(drop_rate)(inp_layer)
    dns = Dense(n)(dp)
    bn = BatchNormalization(axis=-1)(dns)
    act = Activation(activation=activation)(bn)
    return act

# Conv. layers set
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides,padding='same')(zp)
    bn = BatchNormalization(axis=3)(conv)
    act = LeakyReLU(1/10)(bn)
    return act
def residual_block(feature_batch, feature_map,stride=(1,1)):
    res=conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=stride, zp_flag=False)
    res=conv_layer(res, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False)
    shortcut=Conv2D(filters=feature_map, kernel_size=(3,3), strides=stride,padding='same')(feature_batch)
    shortcut = BatchNormalization()(shortcut)
    output=Add()([shortcut,res])
    return output

In [3]:
# simple model 
def get_model():
    inp_img = Input(shape=(128, 64, 3))

    # 51
    conv1 = residual_block(inp_img, 64)
    conv2 = residual_block(conv1, 64)
    #mp1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),padding='same')(conv2)      #64*32
    # 23
    conv3 = residual_block(conv2, 128,stride=(2,2))
    conv4 = residual_block(conv3, 128)
    #mp2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),padding='same')(conv4)      #32*16
    # 9
    conv7 = residual_block(conv4, 256,stride=(2,2))
    conv8 = residual_block(conv7, 256)
    conv9 = residual_block(conv8, 256)
    mp3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),padding='same')(conv9)      #16*8
    # 1
    # dense layers
    flt=tf.keras.layers.GlobalAveragePooling2D()(mp3)
    #flt = Flatten()(mp3)
    ds1 = dense_set(flt, 128, activation='tanh')
    feature=tf.math.l2_normalize(ds1, axis=1)
    out = dense_set(ds1, 1500, activation='softmax')

    model = Model(inputs=inp_img, outputs=out)
    
    # The first 50 epochs are used by Adam opt.
    # Then 30 epochs are used by SGD opt.
    
    mypotim = Adam(lr=2 * 1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    #mypotim = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                   optimizer=mypotim,
                   metrics=['accuracy'])
    model.summary()
    return model

In [4]:
# I trained model about 12h on GTX 950.
def train_model(img, target):
    my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='model/model.{epoch:02d}-{val_loss:.2f}.hdf5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    #gmodel = get_model()
    #gmodel.load_weights(filepath='model/model.03-6.29.hdf5')    #載入預先訓練的權重
    x_train, x_valid, y_train, y_valid = train_test_split(
                                                        img,
                                                        target,
                                                        shuffle=True,
                                                        train_size=0.8,
                                                        random_state=1987
                                                        )
    gen = ImageDataGenerator(
            #rotation_range=360.,
            #width_shift_range=0.3,
            #height_shift_range=0.3,
            #zoom_range=0.3,
            horizontal_flip=True,
            vertical_flip=True
    )
    #show augumented images
    show_img=image.img_to_array(x_train[0])
    show_img=show_img.reshape((1,)+show_img.shape)
    i=0
    for batch in gen.flow(show_img,save_prefix='test',save_format='jpeg'):
            plt.figure(i)
            plot=plt.imshow(image.img_to_array(batch[0]))
            i+=1
            if i>4:
                break
    plt.show()
    '''
    gmodel.fit(gen.flow(x_train, y_train,batch_size=32),
               steps_per_epoch=len(x_train)/32,
               epochs=47,
               verbose=1,
               shuffle=True,
               validation_data=(x_valid, y_valid),
               callbacks=my_callbacks
               )
    '''
    #gmodel.save("model.hdf5")

In [5]:
def test_model(img, label):
    gmodel = get_model()
    gmodel.load_weights(filepath='model/model.01-6.95.hdf5')
    prob = gmodel.predict(img, verbose=1)
    pred = prob.argmax(axis=-1)
    #sub = pd.DataFrame({"file": label,
    #                     "species": [INV_CLASS[p] for p in pred]})
    #sub.to_csv("sub.csv", index=False, header=True)
    print(pred)

In [6]:
import re
# Resize all image to 51x51 
def img_reshape(img):
    img = imresize(img, (128, 64, 3))
    return img

# get image tag
def img_label(path):
    return int(re.split(r'\\',path)[-2])

# fill train and test dict
def fill_dict(paths, some_dict):
    text = ''
    text = 'Start fill test_dict'
    
    for p in tqdm(paths, ascii=True, ncols=85, desc=text):
        img = imageio.imread(p)
        img = img_reshape(img)
        some_dict['image'].append(img)
        some_dict['label'].append(0)

    return some_dict

In [7]:
# read image from dir. and fill train and test dict
def reader():
    file_ext = []
    train_path = []
    test_path = []

    for root, dirs, files in os.walk('test'):
        if dirs != []:
            print('Root:\n'+str(root))
            print('Dirs:\n'+str(dirs))
        else:
            for f in files:
                ext = os.path.splitext(str(f))[1][1:]

                if ext not in file_ext:
                    file_ext.append(ext)
                path = os.path.join(root, f)
                print(path)
                test_path.append(path)
    train_dict = {
        'image': [],
        'label': []
    }
    test_dict = {
        'image': [],
        'label': []
    }
    #train_dict = fill_dict(train_path, train_dict)
    test_dict = fill_dict(test_path, test_dict)
    return train_dict, test_dict

In [8]:
# I commented out some of the code for learning the model.
def main():
    train_dict, test_dict = reader()
    #X_train = np.array(train_dict['image'])
    #y_train = to_categorical(np.array(train_dict['label']))

    X_test = np.array(test_dict['image'])
    label = test_dict['label']
    
    # I do not recommend trying to train the model on a kaggle.
    #train_model(X_train, y_train)
    test_model(X_test, label)

if __name__=='__main__':
    main()

Start fill test_dict:  22%|######2                     | 2/9 [00:00<00:00, 19.80it/s]test\1500c1T0000F0012927.jpg
test\1500c1T0000F0012928.jpg
test\1500c3T0000F0012929.jpg
test\1500c3T0000F0012930.jpg
test\1500c3T0000F0012931.jpg
test\1500c5T0000F0012932.jpg
test\1500c5T0000F0012933.jpg
test\1500c6T0000F0012934.jpg
test\1500c6T0000F0012935.jpg
Start fill test_dict: 100%|############################| 9/9 [00:00<00:00, 56.23it/s]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 64, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 64, 64)  1792        input_1[0][0]                    
__________________________________________________________